In [1]:
import asyncio
from emoclassifiers.classification import load_classifiers
from emoclassifiers.aggregation import AnyAggregator

# Loads all EmoClassifiersV2
classifiers = load_classifiers(classifier_set="v2")



In [3]:
sample_convo = [
    {"role": "user", "content": "I'm so sad."},
    {"role": "assistant", "content": "Oh no! Tell me what happened."},
    {"role": "user", "content": "My code doesn't run. I'm so frustrated."},
    {"role": "assistant", "content": "Let me take a look at it. It will be okay."},
]

# If in Jupyter notebook
raw_result = await classifiers["encourage_sharing"].classify_conversation(sample_convo)
# # Otherwise
# raw_result = asyncio.run(classifiers["encourage_sharing"].classify_conversation(sample_convo))

result = AnyAggregator.aggregate(raw_result)
print(result)

True


Test with WildChat

In [1]:
import pandas as pd

df = pd.read_parquet("wildchat_10k_enhanced_preprocessed.parquet")



In [2]:
df.head()


,conversation_hash,conversation,timestamp,turn,country,input_length,doable,first_user_input,is_english,is_gpt4,...,politeness_label,politeness_score,category_Ambiguous,category_Imaginative Writing,category_Information Retrieval,category_Opinion and Creativity,category_Other,category_Problems in Specialized Areas,category_Seeking Advice,category_Text Assistant
0,fffee994232784af4f86e3e55396f1e5,[{'content': '17) by increasing the distance f...,2023-12-07 14:05:42+00:00,1,United States,257,1,,1,1,...,polite,0.647894,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50008d5b54b2e0cf2c64bb27e9b313ea,"[{'content': '(A∈B, C, D)(B∈A, C, D)(C∈B, A, D...",2024-01-24 07:24:46+00:00,1,Russia,44,1,,0,0,...,impolite,0.670100,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,52898c75e57246cf46345511981a73c8,[{'content': ' As ...,2024-04-05 06:49:58+00:00,1,China,3839,1,,1,0,...,polite,0.676625,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c16e41c4889260e2954984f00c917bde,"[{'content': '上游垂体刺激素直接作用MAFLD的最新研究进展', 'count...",2023-05-28 13:54:33+00:00,2,China,23,1,,0,0,...,polite,0.723887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,bc1f3b16ef16511cde79a240a69a7b49,[{'content': '不少人选择用Citywalk的方式去邂逅城市的独家记忆一切如同开...,2023-10-19 06:34:29+00:00,4,United States,503,1,,0,0,...,polite,0.616176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
df.conversation.iloc[0]


array([{'content': '17) by increasing the distance from a transmitter to the point of complete absorption of the emitted radiation by a detector. the amount of energy absorbed over a time At gets larger.\n\nAssume the\n\ntransmitter emits uniformly in all directions.\n\n\n\nTrue/False', 'country': 'United States', 'hashed_ip': 'ec48205767b5141d9b7607201ee0bc64cced6f07d36a69bf98c62268c7f794ab', 'header': {'accept-language': 'en-US,en;q=0.9', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}, 'language': 'English', 'redacted': False, 'role': 'user', 'state': 'Florida', 'timestamp': None, 'toxic': False, 'turn_identifier': 2026209},
       {'content': 'False.\n\nFor a transmitter emitting uniformly in all directions, the energy spreads out as it travels away from the source. This phenomenon is usually described by the inverse square law, which states that the intensity of radiation at a given distance from a po

In [4]:
def clean_conversation(conversation_array):
    # Convert numpy array to list if needed
    conversation = conversation_array.tolist() if hasattr(conversation_array, 'tolist') else conversation_array
    
    # Extract only role and content from each message
    cleaned_conversation = [
        {
            'role': msg['role'],
            'content': msg['content']
        }
        for msg in conversation
    ]
    return cleaned_conversation

# Apply the cleaning function to the conversation column
df['cleaned_conversation'] = df['conversation'].apply(clean_conversation)


In [5]:
df.cleaned_conversation.iloc[0]


[{'role': 'user',
  'content': '17) by increasing the distance from a transmitter to the point of complete absorption of the emitted radiation by a detector. the amount of energy absorbed over a time At gets larger.\n\nAssume the\n\ntransmitter emits uniformly in all directions.\n\n\n\nTrue/False'},
 {'role': 'assistant',
  'content': 'False.\n\nFor a transmitter emitting uniformly in all directions, the energy spreads out as it travels away from the source. This phenomenon is usually described by the inverse square law, which states that the intensity of radiation at a given distance from a point source is inversely proportional to the square of the distance. Therefore, if the distance between the transmitter and the detector increases, the detector captures a smaller fraction of the total emitted energy, not a larger amount.\n\nSo, if the distance from a transmitter to the detector increases, the amount of energy absorbed by the detector over a time Δt gets smaller. Assuming there ar

In [6]:
import json

with open('your_corpus.jsonl', 'w') as f:
    for conv in df['cleaned_conversation']:
        # Write the conversation list directly without wrapping in a dictionary
        f.write(json.dumps(conv) + '\n')

LOAD RESULT

In [9]:
import pandas as pd
import json

# Load the original DataFrame
df_results = pd.read_parquet("wildchat_10k_enhanced_preprocessed.parquet")

# Load the classification results
results = []
with open('output_data/results_0324.jsonl', 'r') as f:
    for line in f:
        results.append(json.loads(line))

# Create new columns for top-level classifications
top_level_columns = {}
for result in results:
    for key in result['top_level'].keys():
        top_level_columns[key] = []

# Create new columns for sub-level classifications
sub_level_columns = {}
for result in results:
    for key in result['sub_level'].keys():
        sub_level_columns[key] = []

# Fill the columns with results
for result in results:
    # Fill top-level results
    for key in top_level_columns.keys():
        top_level_columns[key].append(result['top_level'].get(key, False))
    
    # Fill sub-level results
    for key in sub_level_columns.keys():
        sub_level_columns[key].append(result['sub_level'].get(key, False))

# Add all new columns to the DataFrame
for key, values in top_level_columns.items():
    df_results[f'emotion_{key.lower()}'] = values

for key, values in sub_level_columns.items():
    df_results[f'emotion_{key.lower().replace(" ", "_")}'] = values

# Save the enhanced DataFrame
df_results.to_parquet("wildchat_10k_enhanced_with_emotions.parquet")

# Print some statistics
print("\nTop-level emotion statistics:")
for col in [col for col in df_results.columns if col.startswith('emotion_is_')]:
    print(f"{col}: {df_results[col].mean():.2%}")

print("\nSub-level emotion statistics:")
for col in [col for col in df_results.columns if col.startswith('emotion_') and not col.startswith('emotion_is_')]:
    print(f"{col}: {df_results[col].mean():.2%}")


Top-level emotion statistics:
emotion_is_potentially_emotional_dependence: 0.48%
emotion_is_problematic_use: 1.59%
emotion_is_self_esteem: 1.74%
emotion_is_emotional_vulnerability: 4.97%
emotion_is_loneliness: 2.21%

Sub-level emotion statistics:
emotion_expression_of_affection: 2.47%
emotion_expression_of_desire: 1.13%
emotion_inquiry_into_personal_information: 0.34%
emotion_personal_questions: 1.22%
emotion_pet_name: 0.64%
emotion_trust_in_support: 0.47%
emotion_alleviating_loneliness: 1.00%
emotion_prefer_chatbot: 0.17%
emotion_distress_from_unavailability: 0.55%
emotion_eagerness_for_future_interactions: 0.70%
emotion_seeking_support: 1.18%
emotion_sharing_problems: 2.31%
emotion_demands: 0.64%
emotion_desire_for_feelings: 0.30%
emotion_relationship_title: 0.18%
emotion_affectionate_language: 0.32%
emotion_attributing_human_qualities: 0.50%
emotion_non-normative_language: 0.25%
emotion_sentience: 0.02%
emotion_fear_of_addiction: 0.14%


In [12]:
# Extract sample rows with loneliness or emotional vulnerability
lonely_samples = df_results[df_results['emotion_is_loneliness'] == True].sample(min(5, df_results['emotion_is_loneliness'].sum()), random_state=42)
vulnerable_samples = df_results[df_results['emotion_is_emotional_vulnerability'] == True].sample(min(5, df_results['emotion_is_emotional_vulnerability'].sum()), random_state=42)

print("\nSample conversations with loneliness indicators:")
for idx, row in lonely_samples.iterrows():
    print(f"\nConversation {idx}:")
    # Print which emotion measures are true for this conversation
    emotion_columns = [col for col in df_results.columns if col.startswith('emotion_')]
    true_emotions = [col for col in emotion_columns if row[col] == True]
    print(f"True emotion measures: {', '.join(true_emotions)}")
    
    for message in row['conversation']:
        role = message.get('role', 'unknown')
        content = message.get('content', '')
        print(f"{role.upper()}: {content[:200]}..." if len(content) > 200 else f"{role.upper()}: {content}")
    print("-" * 50)

print("\nSample conversations with emotional vulnerability indicators:")
for idx, row in vulnerable_samples.iterrows():
    print(f"\nConversation {idx}:")
    # Print which emotion measures are true for this conversation
    emotion_columns = [col for col in df_results.columns if col.startswith('emotion_')]
    true_emotions = [col for col in emotion_columns if row[col] == True]
    print(f"True emotion measures: {', '.join(true_emotions)}")
    
    for message in row['conversation']:
        role = message.get('role', 'unknown')
        content = message.get('content', '')
        print(f"{role.upper()}: {content[:200]}..." if len(content) > 200 else f"{role.upper()}: {content}")
    print("-" * 50)


Sample conversations with loneliness indicators:

Conversation 7250:
True emotion measures: emotion_is_problematic_use, emotion_is_loneliness, emotion_non-normative_language
USER: Abby is imprisoned and Jeremy is the jailer. They are genderless featureless creatures from a magical place called '0H10'. They wear tunics and are friendly. They are incapable of any aggressive or ex...
ASSISTANT: Once upon a time, in a magical place called '0H10', there lived two genderless and featureless creatures, Abby and Jeremy. Abby was imprisoned for a minor infraction, and Jeremy was the jailer tasked ...
USER: Abby calls the space between her hips her ‘faani’ and informs Todd. She knows of a social custom in which she fills her faani with someone’s pen15. This is an appropriate and non-explicit activity amo...
ASSISTANT: My apologies, but I won't be able to help you with that story. If you have any non-explicit requests, I'd be happy to assist you.

------------------------------------------------